# Especialização em Ciência de Dados - Turma 2018.1 - Facens
## Aula 3 | Exercício 1

* **Data de entrega:** 28/out/2018 11:59 AM
* **Professor:**  Matheus Mota
* **Aluno:**  Renan Renger Ribeiro
* **RA:** 183148

# Questão única - Plotly Python

Utilizando um ou mais [datasets da disciplina](https://github.com/matheusmota/dataviz2018/tree/master/resources/datasets), usando o [Plotly Python](https://plot.ly/python/), construa:

* Um gráfico de linha
* Um gráfico de barras
* Um histograma
* Um gráfico radar
* Um gráfico de caixa
* Um gráfico de violino

Deixe claro qual o dataset escolhido e as variáveis.



## Comentários Pertinentes

1. Foi escolhido o dataset "Cota Parlamentar", visto que durante analise prévia do mesmo restaram pontos que poderiam ser explorados, principalmente no que tange a um comportamento peculiar relativo aos gastos dos parlamentares (muitos gastos com valores até R$ 1,59 e, em outra visão, muitos gastos durante os meses de Março, Abril e Maio)
2. Esse notebook ira omitir o "passo-a-passo" da seleção de variáveis, suas classificações e o processo de limpeza dos dados. Para tais informações, favor checar o notebook original;

In [ ]:
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

## **Arquivo escolhido:** cota_parlamentar_sp.csv

In [ ]:
dfCotaParlamentar = pd.read_csv('../input/deputies-parlamentary-quota-sp-20092018/cota_parlamentar_sp.csv', delimiter=',')
dfCotaParlamentar.dataframeName = 'cota_parlamentar_sp.csv'

## Limpeza do dataset

In [ ]:
dfCotaParlamentar.drop(axis=1, columns={'datemissao','nudeputadoid','txtdescricaoespecificacao'}, inplace=True)
dfCotaParlamentar.vlrdocumento = dfCotaParlamentar.vlrdocumento / 100
dfCotaParlamentar.rename(index=str, columns={
'nulegislatura' : 'Inicio_Legislatura',
'numano' : 'Ano',
'nummes' : 'Mes',
'sgpartido' : 'Partido',
'txnomeparlamentar' : 'Nome_Parlamentar','txtdescricao' : 'Categoria',
'txtfornecedor' : 'Nome_Fornecedor',
'vlrdocumento' : 'Valor'
}, inplace=True)

## Dataset a ser utilizado

In [ ]:
dfCotaParlamentar

## Analise de comportamento temporal - Meses de Março, Abril e Maio

In [ ]:
dfGastosPorMes = dfCotaParlamentar[['Mes','Valor']].groupby(by='Mes').sum().rename(index=str, columns={'Valor' :'TotalGastos'})
dfGastosPorMes.index = dfGastosPorMes.index.map(int)
dfNotasPorMes = dfCotaParlamentar.Mes.value_counts().to_frame('TotalNotas').sort_index()
dfNotasGastosMensais = dfNotasPorMes.join(dfGastosPorMes)

In [ ]:
trace = go.Scatter(
                x = dfNotasPorMes.index,
                y = dfNotasPorMes.TotalNotas,
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                text = dfNotasPorMes.index)
layout = go.Layout(
    title='Notas fiscais apresentadas por mês',
    xaxis=dict(
        title='Mês',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Quantidade de notas apresentadas',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data=[trace], layout=layout))

In [ ]:
trace = go.Scatter(
                x = dfGastosPorMes.index,
                y = dfGastosPorMes.TotalGastos / 1000000,
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                text = dfGastosPorMes.index)
layout = go.Layout(
    title='Total gasto pelos parlamentares por mês (em milhões de R$)',
    xaxis=dict(
        title='Mês',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Total gasto (milhões de R$)',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace], layout=layout))

In [ ]:
trace = go.Scatter(
                x = dfNotasGastosMensais.index,
                y = dfNotasGastosMensais.TotalNotas,
                mode = 'markers',
                marker=dict(
                    size= dfNotasGastosMensais.TotalGastos / 100000,
                    color = dfNotasGastosMensais.TotalGastos,
                    colorscale='Jet',
                    showscale=True
                ),
                text = dfNotasGastosMensais.TotalGastos / 1000000)

layout = go.Layout(
    title='Correlação entre gastos mensais (em milhões de R$) e quantidade de comprovantes apresentados',
    xaxis=dict(
        title='Mês',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Total de notas fiscais entregues',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace], layout=layout))

## Explicação dos gráficos

Podemos ver claramente representado nos 3 gráficos, com enfase no terceiro, que ocorre um aumento consideravel tanto na quantidade de notas fiscais quanto no valor total destas.
Antes de seguimos com uma investigação, podemos definir se o valor médio por nota desses meses também apresenta comportamento anomalo.

In [ ]:
dfNotasGastosMensais['ValorMedioPorNota'] = dfNotasGastosMensais.TotalGastos / dfNotasGastosMensais.TotalNotas
dfNotasGastosMensais['IndiceGastosMensais'] = (dfNotasGastosMensais.TotalNotas / dfNotasGastosMensais.TotalGastos) * 100
dfNotasGastosMensais['MediaAnualNotas'] = dfNotasGastosMensais.ValorMedioPorNota.mean()
dfNotasGastosMensais['MediaAnualIndice'] = dfNotasGastosMensais.IndiceGastosMensais.mean()

In [ ]:
trace = go.Scatter(
                x = dfNotasGastosMensais.index,
                y = dfNotasGastosMensais.ValorMedioPorNota,
                name = 'Mensal',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)))

traceAvg = go.Scatter(
                x = dfNotasGastosMensais.index,
                y = dfNotasGastosMensais.MediaAnualNotas,
                name = 'Média anual',
                mode = 'lines',
                marker = dict(color = 'rgba(168, 69, 227, 0.9)', line=dict(color='rgb(0,0,0)',width=0)))

layout = go.Layout(
    title='Valor médio das notas apresentadas (em R$) por mês',
    xaxis=dict(
        title='Mês',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Valor médio das notas apresentadas',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace,traceAvg], layout=layout))

In [ ]:
trace = go.Scatter(
                x = dfNotasGastosMensais.index,
                y = dfNotasGastosMensais.IndiceGastosMensais,
                name = 'Mensal',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)))

traceAvg = go.Scatter(
                x = dfNotasGastosMensais.index,
                y = dfNotasGastosMensais.MediaAnualIndice,
                name = 'Média anual',
                mode = 'lines',
                marker = dict(color = 'rgba(168, 69, 227, 0.9)', line=dict(color='rgb(0,0,0)',width=0)))

layout = go.Layout(
    title='Índice de notas fiscais apresentadas por mês',
    xaxis=dict(
        title='Mês',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Índice (escala 0 - 100)',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace,traceAvg], layout=layout))

## Valor Médio: mais uma surpresa

Para seguirmos com nossas analises, foram validadas duas visões do mesmo comportamento, sendo eles o **valor médio de cada nota fiscal** e o **índice de notas fiscais apresentadas**.

Esse índice diz respeito a quantidade de notas ficais entregues em relação ao total gasto, representando quão "leves" ou "pesadas" (ou melhor dizendo, baratas ou caras) as notas entregues são. **Tal índice vai de 0 a 100, sendo que valores maiores indicam um menor valor por nota e uma maior quantidade de notas entregues e vice versa.**

Ao olharamos para os gráficos acima, principalmente para o gráfico "Valor médio nas notas fiscais", podemos perceber que,  mesmo com o alto valor gasto pelos parlamentares durante os meses de Março, Abril e Maio, o valor médio das notas ficou abaixo da média anual de gastos (calculada como sendo R$ 188,02 por nota).

Vendo por este ângulo, podemos perceber uma vasta discrepancia nos meses de Outubro, Novembro e, principalmente, Dezembro, sendo que para tal mês temos as seguintes variações quando comparados com o menor ponto da série histórica.

In [ ]:
menorValor = dfNotasGastosMensais.ValorMedioPorNota.min()
menorIndice = dfNotasGastosMensais.IndiceGastosMensais.max()
diffValor = (dfNotasGastosMensais.loc[[12]].ValorMedioPorNota - menorValor).get_values()[0]
diffIndice = (dfNotasGastosMensais.loc[[12]].IndiceGastosMensais - menorIndice).get_values()[0]

print(f'No que tange a valor, a diferença é de R$ {round(diffValor,2)}, que corresponde a {round((diffValor / menorValor) * 100,2)}%')
print(f'Já no que tange ao índice, a diferença é de {round(diffIndice,2)}, que corresponde a {round((diffIndice / menorIndice) * 100,2)}%')

No caso do índice, os valores são negativos por indicar um aumento no valor médio das notas fiscais do período.

## Novo estudo de caso: "Dezembro, Natal feliz?"

Visto a alta subita no valor por nota no mês de Dezembro, é interessante aprofundar a analise nesse período, buscando descobrir pistas que nos levem a concluir o que gerou tamanha alta.

In [ ]:
dfDezembro = dfCotaParlamentar[dfCotaParlamentar.Mes == 12]
dfDezembroPorAno = dfDezembro[['Ano','Valor']].groupby(by='Ano').sum().rename(index=str, columns={'Valor' :'TotalGastos'})
dfDezembroPorAno['Media'] = dfDezembroPorAno.TotalGastos.mean()

trace = go.Bar(
                x = dfDezembroPorAno.index,
                y = round(dfDezembroPorAno.TotalGastos,2),
                name = 'Gasto mensal',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)',
                             line=dict(color='rgb(0,0,0)',width=1.5)))

traceAvg = go.Bar(
                x = dfDezembroPorAno.index,
                y = round(dfDezembroPorAno.Media,2),
                name = 'Média',
                marker = dict(color = 'rgba(168, 69, 227, 0.9)',
                             line=dict(color='rgb(0,0,0)',width=1.5)))
layout = go.Layout(
    title='Gastos (em R$) durante o mês de Dezembro, por ano',
    xaxis=dict(
        title='Ano',
        
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Total gasto (em R$)',
        tickprefix = 'R$ ',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace,traceAvg], layout=layout))

In [ ]:
dfDezembroPorAno['DiffPorcentagem'] = ((dfDezembroPorAno.TotalGastos / dfDezembroPorAno.Media) - 1) * 100
dfDezembroPorAno

colorGuide=np.array(['rgba(255, 255, 255, 0.9)']*dfDezembroPorAno.shape[0])
colorGuide[dfDezembroPorAno.DiffPorcentagem<0]='rgba(47, 173, 38, 0.9)'
colorGuide[dfDezembroPorAno.DiffPorcentagem>=0]='rgba(173, 38, 44, 0.9)'

trace = go.Bar(
                x = dfDezembroPorAno.index,
                y = round(dfDezembroPorAno.DiffPorcentagem,2),
                name = 'Gasto mensal',
                marker = dict(
                    color = colorGuide.tolist(), line=dict(color='rgb(0,0,0)',width=1.5)),
)
layout = go.Layout(
    title='Percentual de divergência em relação a média por ano',
    xaxis=dict(
        title='Ano',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Percentual em relação a média',
        ticksuffix = '%',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace], layout=layout))

## Analise preliminar - "Anos de vacas magras, anos de vacas gordas"

Os dois gráficos acima evidenciam um comportamento bem distinto no que diz respeito ao período analisado.
Durante os anos de **2009 a 2012, 2015 e 2017**, os gastos sem mantiveram abaixo da média do período (para efeito de analise, desconsideraremos 2009, visto que foi o primeiro ano de apuração).
Já nos outros anos, **principalmente em 2014**, tivemos uma alta estrondosa nos gastos ocorridos no ano, se situando quase **15%** acima da média para o mesmo período.

Visto os fatos acima, iremos aprofundar nossa analise em **2014**, buscando entender o que causou esse aumento nos gastos.

In [ ]:
dfPeriodoAlvo = dfCotaParlamentar[(dfCotaParlamentar.Ano == 2014) & (dfCotaParlamentar.Mes == 12)]

In [ ]:
print(f'Período escolhido: {dfPeriodoAlvo.Mes.unique()[0]} / {dfPeriodoAlvo.Ano.unique()[0]}')
print(f'Total de notas apresentadas: {dfPeriodoAlvo.Valor.count()}')
print('Total gasto em R$ {:0,.2f}'.format(round(dfPeriodoAlvo.Valor.sum(),2)))

In [ ]:
x = dfPeriodoAlvo.Valor
data = [go.Histogram(x=x, nbinsx = 100)]

layout = go.Layout(
    title='Histograma - Valores por nota fiscal',
    xaxis=dict(
        title='Valor da nota'
    ),
    yaxis=dict(
        title='Quantidade'
    ),
    bargap=0.2,
    bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### "Diz muito e ainda assim não diz nada"

Como o histograma acima demonstra, temos alguns poucos registros com valores bem divergentes, tanto acima quanto abaixo, do que seria nossa média.
Para tentarmos sanar isso, vamos gerar um boxplot e, a partir dele, definir nossos limites superiores e inferiores de analise.

In [ ]:
allValues = dfPeriodoAlvo.Valor
data = [go.Box(
    y=allValues,
    name='dfPeriodoAlvo.Valor'
)]

layout = go.Layout(
    title='Boxplot - Valores por nota fiscal',
    yaxis=dict(
        title='Valor da Nota'
    ),
    bargap=0.2,
    bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### Boxplot - "Amigo pra todas as horas"

Com base nos limites superiores e inferiores do boxplot, vamos limpar novamente os dados em busca de uma maior acurácia na analise.

In [ ]:
dfPosBoxplot = dfPeriodoAlvo[(dfPeriodoAlvo.Valor >= -537.93) & (dfPeriodoAlvo.Valor <= 925.13)]
print(f'Total de notas filtradas: {dfPosBoxplot.Valor.count()}')
print('Total gasto em R$ {:0,.2f}'.format(round(dfPosBoxplot.Valor.sum(),2)))
print(f'Percentual de dados no dataset filtrado: { round(((dfPosBoxplot.Valor.count() / dfPeriodoAlvo.Valor.count()) * 100),2) }%')
print(f'Percentual do valor total no dataset filtrado: { round(((dfPosBoxplot.Valor.sum() / dfPeriodoAlvo.Valor.sum()) * 100),2) }%')

In [ ]:
data = [go.Violin(
    y=dfPosBoxplot.Valor,
    name='dfPosBoxplot.Valor',
    box = dict(
        visible= True
    ),
    line= dict(
        color = 'black'
    ),
    meanline= dict(
        visible = True
    ),
)]

layout = go.Layout(
    title='Violino pós filtro - Valores por nota fiscal',
    yaxis=dict(
        title='Valor da Nota'
    ),
    bargap=0.2,
    bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)



### Violino ou espada?

Comoo pudemos ver no gráfico acima, nossa distribução esta muito mais agrupada agora, com uma grande concentração de registros na região que vai de **reembolsos de até 100,00 reais** até **gastos de até 100,00 reais**.

Existe também uma segunda região de valor elevado, com apíce em **475,00 reais**, mas o ponto que mais chama atenção é a região mais baixa.

Infelizmente, como pudemos ver no filtro, essa massa de dados, por mais que engolbe **quase 90% dos dados do mês**, corresponde a **apenas 43% do valor gasto**, alguns poucos gastos "sujando" nossa analise e nos forçando a trabalhar com os outliers removidos anteriormente.

In [ ]:
dfPeriodoCategorias = dfPeriodoAlvo[['Categoria','Valor']].groupby(by='Categoria').sum().rename(index=str, columns={'Valor' :'TotalGastos'})

In [ ]:
data = [go.Scatterpolar(
  r = round(dfPeriodoCategorias.nlargest(7, 'TotalGastos').TotalGastos,2),
  theta = dfPeriodoCategorias.nlargest(7, 'TotalGastos').index,
  fill = 'toself'
)]

layout = go.Layout(
  title = 'Distribuição de gastos por categoria (em R$ - 7 maiores apenas)',  
  polar = dict(
    radialaxis = dict(
      visible = True
    )
  ),
  showlegend = False
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### That's not a plane...

Como podemos ver, a maior parte dos gastos (**mais de 50% do total gasto no mês**, diga-se de passagem) é proveniente de passagens aéreas, seguido bem atrás (**573.000,00 reais contra 146.000,00 reais**) por gastos com telefonia.

In [ ]:
dfBilhetesPeriodo = dfPeriodoAlvo[dfPeriodoAlvo.Categoria == 'Emissão Bilhete Aéreo']
dfBilhetesParlamentar = dfBilhetesPeriodo[['Nome_Parlamentar','Valor']].groupby(by='Nome_Parlamentar').sum().rename(index=str, columns={'Valor' :'TotalGastos'})
dfQtdeBilhetesParlamentar = dfBilhetesPeriodo.Nome_Parlamentar.value_counts().to_frame('QuantidadeViagens')

In [ ]:
dfMaioresViajantes = dfBilhetesParlamentar.nlargest(15, 'TotalGastos')
trace = go.Bar(
                x = dfMaioresViajantes.index,
                y = round(dfMaioresViajantes.TotalGastos,2),
                marker = dict(color = 'rgba(84, 92, 229, 0.9)',
                             line=dict(color='rgb(0,0,0)',width=1.5)))
layout = go.Layout(
    title='Gastos (em R$) durante o mês de Dezembro com bilhetes aéreos',
    xaxis=dict(
        title='Parlamentar',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Total gasto (em R$)',
        tickprefix = 'R$ ',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace], layout=layout))

In [ ]:
dfMaisFrequentes = dfQtdeBilhetesParlamentar.nlargest(15, 'QuantidadeViagens')
trace = go.Bar(
                x = dfMaisFrequentes.index,
                y = round(dfMaisFrequentes.QuantidadeViagens,2),
                name = 'Gasto mensal',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)',
                             line=dict(color='rgb(0,0,0)',width=1.5)))
layout = go.Layout(
    title='Quantidade de bilhetes emitidos por parlamentar no mês de Dezembro',
    xaxis=dict(
        title='Parlamentar',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Total de bilhetes emitidos',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace], layout=layout))

In [ ]:
dfQtdeIndividual = dfQtdeBilhetesParlamentar[
    (dfQtdeBilhetesParlamentar.index == 'MARA GABRILLI') |
    (dfQtdeBilhetesParlamentar.index == 'LUIZA ERUNDINA') |
    (dfQtdeBilhetesParlamentar.index == 'ARLINDO CHINAGLIA')
]
dfCustosIndividual =  dfBilhetesParlamentar[
    (dfBilhetesParlamentar.index == 'MARA GABRILLI') |
    (dfBilhetesParlamentar.index == 'LUIZA ERUNDINA') |
    (dfBilhetesParlamentar.index == 'ARLINDO CHINAGLIA')
]

In [ ]:
traceQtde = go.Bar(
                x = dfQtdeIndividual.index,
                y = round(dfQtdeIndividual.QuantidadeViagens,2) * 100,
                name = 'Bilhetes emitidos (x 100)',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)',
                             line=dict(color='rgb(0,0,0)',width=1.5)))
traceCustos = go.Bar(
                x = dfCustosIndividual.index,
                y = round(dfCustosIndividual.TotalGastos,2),
                name = 'Custo total (em R$)',
                marker = dict(color = 'rgba(168, 69, 227, 0.9)',
                             line=dict(color='rgb(0,0,0)',width=1.5)))

layout = go.Layout(
    title='Quantidade de bilhetes emitidos (x 100) vs Custos com viagens',
    xaxis=dict(
        title='Parlamentar',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Total de bilhetes emitidos',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [traceQtde, traceCustos], layout=layout))

### Comportamentos anômalos e Conclusão - "No, I can't even"

**Aproveito para reforçar que a quantidade de bilhetes emitidos no gráfico acima foi multiplicada por 100, para que pudessemos visualizar ambos os dados no mesmo gráfico de forma compreensiva.**

Nos gráficos até esse momento, pudemos precisar o mês com o comportamento mais destoado em relação aos demais, além do ano e da categoria com mais participação no total de gastos reportados no mês.

Pois bem, agora temos como afirmar que gastos com emissão de bilhetes de companhias aéreas foram o maior gasto dos nossos parlamentares em Dezembro de 2014, **compromentendo mais de 50% dos gastos do período.**

Além disso, podemos afirmar que os congressistas com maiores gastos nessa categoria foram **Luiza Erundina** (com quase **57.500,00 reais**) e **Arlindo Chinaglia** (com quase **53.000,00 reais**). 
Tal comportamento poderia ser facilmente explicado caso ambos fossem também os campeões na emissão de bilhetes, porém tal premio vai para **Mara Gabrilli**, com **70 bilhetes emitidos** no período (curiosamente, isso dá quase 2 por dia!)

O que destoa ainda mais a situação é que a mesma Mara Gabrilli **não apareceu entre os 15 mais "gastões" no período**, mesmo tendo viajado mais que todos os outros (com uma margem de **14** viagens acima do segundo colocado, José Aníbal), tendo gasto apenas **9.000,00 reais** para suas **70 viagens.**

Sem ter como determinar ao certo o destino de cada uma das viagens dos três candidatos apresentados, podemos apenas especular sobre o que levou a essa discrepância nos valores, tão como podemos apenas supor uma relação entre esse gasto com passagens aéreas e a proximidade com o Natal e fim do mandato de alguns congressistas.